In [9]:
#1. get xmlboxes
#2. get xmlboxes img region from bkg
#3. put the img region to new image
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'
os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir


In [10]:
def past_to_background_from_image_file(file, bboxes, background_img_array, save_name, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S.%f")
    cv2.imwrite(GENE_IMG_DIR + time_str + '_' + save_name.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    print(xml_name)       
    if(not (int(xml_name[:-4]) >= 442000)):
        continue
    background_img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    past_to_background_from_image_file(SEED_BKG_DIR + seed_bkg_names[5], bboxes, background_img_data, 
                                       SEED_IMG_DIR + xml_name[:-3] + 'jpg')

00000442000.xml
00000442001.xml
00000442002.xml
00000442003.xml
00000442004.xml
00000442005.xml
00000442006.xml
00000442007.xml
00000442008.xml
00000442009.xml
00000442010.xml
00000442011.xml
00000442012.xml
00000442013.xml
00000442014.xml
00000442015.xml
00000442016.xml
00000442017.xml
00000442018.xml
00000442019.xml
00000442020.xml
00000442021.xml
00000442022.xml
00000442023.xml
00000442024.xml
00000442025.xml
00000442026.xml
00000442027.xml
00000442028.xml
00000442029.xml
00000442030.xml
00000442031.xml
00000442032.xml
00000442033.xml
00000442034.xml
00000442035.xml
00000442036.xml
00000442037.xml
00000442038.xml
00000442039.xml
00000442040.xml
00000442041.xml
00000442042.xml
00000442043.xml
00000442044.xml
00000442045.xml
00000442046.xml
00000442047.xml
00000442048.xml
00000442049.xml
00000442050.xml
00000442051.xml
00000442052.xml
00000442053.xml
00000442054.xml
00000442055.xml
00000442056.xml
00000442057.xml
00000442058.xml
00000442059.xml
00000442060.xml
00000442061.xml
00000442

00000442525.xml
00000442526.xml
00000442527.xml
00000442528.xml
00000442529.xml
00000442530.xml
00000442531.xml
00000442532.xml
00000442533.xml
00000442534.xml
00000442535.xml
00000442536.xml
00000442537.xml
00000442538.xml
00000442539.xml
00000442540.xml
00000442541.xml
00000442542.xml
00000442543.xml
00000442544.xml
00000442545.xml
00000442546.xml
00000442547.xml
00000442548.xml
00000442549.xml
00000442550.xml
00000442551.xml
00000442552.xml
00000442553.xml
00000442554.xml
00000442555.xml
00000442556.xml
00000442557.xml
00000442558.xml
00000442559.xml
00000442560.xml
00000442561.xml
00000442562.xml
00000442563.xml
00000442564.xml
00000442565.xml
00000442566.xml
00000442567.xml
00000442568.xml
00000442569.xml
00000442570.xml
00000442571.xml
00000442572.xml
00000442573.xml
00000442574.xml
00000442575.xml
00000442576.xml
00000442577.xml
00000442578.xml
00000442579.xml
00000442580.xml
00000442581.xml
00000442582.xml
00000442583.xml
00000442584.xml
00000442585.xml
00000442586.xml
00000442

00000443047.xml
00000443048.xml
00000443049.xml
00000443050.xml
00000443051.xml
00000443052.xml
00000443053.xml
00000443054.xml
00000443055.xml
00000443056.xml
00000443057.xml
00000443058.xml
00000443059.xml
00000443060.xml
00000443061.xml
00000443062.xml
00000443063.xml
00000443064.xml
00000443065.xml
00000443066.xml
00000443067.xml
00000443068.xml
00000443069.xml
00000443070.xml
00000443071.xml
00000443072.xml
00000443073.xml
00000443074.xml
00000443075.xml
00000443076.xml
00000443077.xml
00000443078.xml
00000443079.xml
00000443080.xml
00000443081.xml
00000443082.xml
00000443083.xml
00000443084.xml
00000443085.xml
00000443086.xml
00000443087.xml
00000443088.xml
00000443089.xml
00000443090.xml
00000443091.xml
00000443092.xml
00000443093.xml
00000443094.xml
00000443095.xml
00000443096.xml
00000443097.xml
00000443098.xml
00000443099.xml
00000443100.xml
00000443101.xml
00000443102.xml
00000443103.xml
00000443104.xml
00000443105.xml
00000443106.xml
00000443107.xml
00000443108.xml
00000443

00000443566.xml
00000443567.xml
00000443568.xml
00000443569.xml
00000443570.xml
00000443571.xml
00000443572.xml
00000443573.xml
00000443574.xml
00000443575.xml
00000443576.xml
00000443577.xml
00000443578.xml
00000443579.xml
00000443580.xml
00000443581.xml
00000443582.xml
00000443583.xml
00000443584.xml
00000443585.xml
00000443586.xml
00000443587.xml
00000443588.xml
00000443589.xml
00000443590.xml
00000443591.xml
00000443592.xml
00000443593.xml
00000443594.xml
00000443595.xml
00000443596.xml
00000443597.xml
00000443598.xml
00000443599.xml
00000443600.xml
00000443601.xml
00000443602.xml
00000443603.xml
00000443604.xml
00000443605.xml
00000443606.xml
00000443607.xml
00000443608.xml
00000443609.xml
00000443610.xml
00000443611.xml
00000443612.xml
00000443613.xml
00000443614.xml
00000443615.xml
00000443616.xml
00000443617.xml
00000443618.xml
00000443619.xml
00000443620.xml
00000443621.xml
00000443622.xml
00000443623.xml
00000443624.xml
00000443625.xml
00000443626.xml
00000443627.xml
00000443